In [1]:
%matplotlib inline

In [2]:
!pip install pytorch_lightning -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.2/826.2 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 8.8 MB/s eta 0:00:00



[Learn the Basics](intro.html) ||
[Quickstart](quickstart_tutorial.html) ||
[Tensors](tensorqs_tutorial.html) ||
[Datasets & DataLoaders](data_tutorial.html) ||
[Transforms](transforms_tutorial.html) ||
[Build Model](buildmodel_tutorial.html) ||
[Autograd](autogradqs_tutorial.html) ||
**Optimization** ||
[Save & Load Model](saveloadrun_tutorial.html)

# Optimizing Model Parameters

Now that we have a model and data it's time to train, validate and test our model by optimizing its parameters on
our data. Training a model is an iterative process; in each iteration the model makes a guess about the output, calculates
the error in its guess (*loss*), collects the derivatives of the error with respect to its parameters (as we saw in
the [previous section](autograd_tutorial.html)), and **optimizes** these parameters using gradient descent. For a more
detailed walkthrough of this process, check out this video on [backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8)_.

## Prerequisite Code
We load the code from the previous sections on [Datasets & DataLoaders](data_tutorial.html)
and [Build Model](buildmodel_tutorial.html).


In [3]:
import torch
import os
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torchvision.transforms import ToTensor
import pytorch_lightning as pl

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

learning_rate = 1e-3
batch_size = 64
epochs = 10

class LitClassifier(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1)
        pred = self.model(x)
        loss = F.cross_entropy(pred, y)
        self.log("train_loss", loss)
        return {'loss': loss}

    def test_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1)
        pred = self.model(x)
        loss = F.cross_entropy(pred, y)
        self.log("test_loss", loss)
        return {'test_loss': loss}

    # def validation_step(self, val_batch, batch_idx):
    #     x, y = val_batch
    #     x = x.view(x.size(0), -1)
    #     pred = self.model(x)
    #     loss = F.cross_entropy(pred, y)
    #     self.log("val_loss", loss)

    #     return {'val_loss': loss}

    # def validation_epoch_end(self, outputs):
    #     # outputs is an array with what you returned in validation_step for each batch
    #     # outputs = [{'loss': batch_0_loss}, {'loss': batch_1_loss}, ..., {'loss': batch_n_loss}]
        
    #     avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
    #     self.log("avg_val_loss", avg_loss)

    #     return {'avg_val_loss': avg_loss}

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=learning_rate)
        return optimizer

    def prepare_data(self):
    # prepare transforms standard to MNIST
        FashionMNIST(os.getcwd(), train=True, download=True)
        FashionMNIST(os.getcwd(), train=False, download=True)

    def train_dataloader(self):
        mnist_train = FashionMNIST(os.getcwd(), train=True, download=False, 
                            transform=transforms.ToTensor())
        # self.mnist_train, self.mnist_val = random_split(mnist_train, [55000, 5000])

        mnist_train = DataLoader(mnist_train, batch_size=batch_size)
        return mnist_train
    # def val_dataloader(self):
    #     mnist_val = DataLoader(self.mnist_val, batch_size=batch_size)
    #     return mnist_val

    def test_dataloader(self):
        mnist_test = FashionMNIST(os.getcwd(), train=False, download=False, 
                          transform=transforms.ToTensor())
        mnist_test = DataLoader(mnist_test, batch_size=batch_size)
        return mnist_test

In [5]:
# model
mymodel = LitClassifier(NeuralNetwork())

# train model
mymodel.prepare_data()
mymodel.train_dataloader()
#Change the GPU number to the number of gpus you wish to use
trainer = pl.Trainer(max_epochs = epochs,accelerator="gpu", devices=1)
trainer.fit(model=mymodel)
trainer.test(model=mymodel)
print("Done!")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type          | Params
----------------------------------------
0 | model | NeuralNetwork | 669 K 
----------------------------------------
669 K     Trainable params
0         Non-trainable params
669 K     Total params
2.679     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.7940288186073303
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Done!


## Further Reading
- [Loss Functions](https://pytorch.org/docs/stable/nn.html#loss-functions)
- [torch.optim](https://pytorch.org/docs/stable/optim.html)
- [Warmstart Training a Model](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)


